In [14]:
# calling requests with python
import requests
import numpy as np
import pandas as pd

In [15]:
# call this request to get the data

url = 'https://api.globalsolaratlas.info/data/lta?loc=22.917923,50.449219'
response = requests.get(url)
print(response.status_code)
print(response.text)


200
{"annual":{"metadata":{"ts":1707680632029,"version":{"data":"1.3","modules":{"@solargis/prospect-service":"9.8.0"}},"layers":{"PVOUT_csi":{"method":"PV simulation model","period":{"from":"satregion","to":"2022"},"sources":["GHI, DNI, TEMP, OPTA, ALBEDO, ELE (Solargis)"],"updated":"2023-04-01","version":"2.2.26","unit":"kWh/kWp"},"DNI":{"method":"Solar model","period":{"from":"satregion","to":"2022"},"sources":["Solargis solar model (Solargis)"],"updated":"2023-04-01","version":"2.2.26","unit":"kWh/m2"},"GHI":{"method":"Solar model","period":{"from":"satregion","to":"2022"},"sources":["Solargis solar model (Solargis)"],"updated":"2023-04-01","version":"2.2.26","unit":"kWh/m2"},"DIF":{"method":"Solar model","period":{"from":"satregion","to":"2022"},"sources":["Solargis solar model (Solargis)"],"updated":"2023-04-01","version":"2.2.26","unit":"kWh/m2"},"GTI_opta":{"method":"Solar model","period":{"from":"satregion","to":"2021"},"sources":["DNI, GHI, OPTA, ALBEDO, ELE (Solargis)"],"upd

In [16]:
# convert the response to json
resp = response.json()


In [17]:
resp['annual']['data']

{'PVOUT_csi': 1838.9248046875,
 'DNI': 1928.1875,
 'GHI': 2228.1796875,
 'DIF': 900.5859375,
 'GTI_opta': 2419.9296875,
 'OPTA': 27,
 'TEMP': 28.1875,
 'ELE': 138}

In [18]:
vars = list(resp['annual']['metadata']['layers'].keys())
units = [resp['annual']['metadata']['layers'][key]['unit'] for key in resp['annual']['metadata']['layers'].keys()]
col_names = [f'{vars[i]} ({units[i]})' for i in range(len(vars))]

In [19]:
col_names

['PVOUT_csi (kWh/kWp)',
 'DNI (kWh/m2)',
 'GHI (kWh/m2)',
 'DIF (kWh/m2)',
 'GTI_opta (kWh/m2)',
 'OPTA (°)',
 'TEMP (°C)',
 'ELE (m)']

In [40]:
def getData(lat, long):
    url = 'https://api.globalsolaratlas.info/data/lta?loc='
    print(f'Requesting for lattitude = {lat} and longitude = {long}')
    response = requests.get(url+str(lat)+','+str(long))
    resp = response.json()

    return resp['annual']['data']

In [21]:
getData(50,50)

Requesting for lattitude = 50 and longitude = 50


{'PVOUT_csi': 1289.0655517578125,
 'DNI': 1357.25,
 'GHI': 1299.5,
 'DIF': 553.7265625,
 'GTI_opta': 1547.7890625,
 'OPTA': 38,
 'TEMP': 8.1875,
 'ELE': 2}

In [22]:
def dataToList(data):
    return [data[key] for key in vars]

In [23]:
lats, longs = [i for i in range(30,50,10)], [i for i in range(30,50,10)]

In [24]:
data = [dataToList(getData(lat, long)) for lat in lats for long in longs]

Requesting for lattitude = 30 and longitude = 30
Requesting for lattitude = 30 and longitude = 40
Requesting for lattitude = 40 and longitude = 30
Requesting for lattitude = 40 and longitude = 40


In [25]:
df = pd.DataFrame(data=data, index=[f'{lat},{long}' for lat in lats for long in longs], columns= col_names)
df

,PVOUT_csi (kWh/kWp),DNI (kWh/m2),GHI (kWh/m2),DIF (kWh/m2),GTI_opta (kWh/m2),OPTA (°),TEMP (°C),ELE (m)
"30,30",1890.694336,2181.968750,2153.773438,721.195312,2410.320312,31,21.0000,138
"30,40",1931.229736,2371.742188,2216.742188,694.976562,2488.187500,31,22.6875,650
"40,30",1391.344482,1440.851562,1512.781250,627.492188,1717.281250,32,12.6875,726
"40,40",1631.380127,1930.742188,1711.343750,551.523438,1982.210938,35,5.9375,1988


In [26]:
import threading

# example function 
def my_function(arg,arg1):
    print("the sume is :", arg+arg1)

# Define the function you want to run in parallel
def run_parallel(func, args_list):
    threads = []
    
    # Create a thread for each argument
    for args in args_list:
        thread = threading.Thread(target=func, args=args)
        threads.append(thread)
        thread.start()
    
    # Wait for all threads to complete
    for thread in threads:
        thread.join()

# Example usage
args_list = [(1,2), (2,3), (3,4)]  # List of arguments for my_function
run_parallel(my_function, args_list)

the sume is : 3
the sume is : 5
the sume is : 7


In [44]:
div = 1
lats = [i/div for i in range(8*div,37*div,1)]
longs = [i/div for i in range(69*div,96*div,1)]

df = pd.DataFrame(columns= vars)

print(col_names)

def collateData(lat, long):
    data = getData(lat, long)
    # temp = pd.DataFrame(data, index=[f'{lat},{long}'], columns= vars)
    print(f'{lat},{long},',dataToList(data))

args_list = []
for lat in lats:
    for long in longs:
        args_list.append((lat,long))


run_parallel(collateData, args_list)

df

['PVOUT_csi (kWh/kWp)', 'DNI (kWh/m2)', 'GHI (kWh/m2)', 'DIF (kWh/m2)', 'GTI_opta (kWh/m2)', 'OPTA (°)', 'TEMP (°C)', 'ELE (m)']
Requesting for lattitude = 8.0 and longitude = 69.0
Requesting for lattitude = 8.0 and longitude = 70.0
Requesting for lattitude = 8.0 and longitude = 71.0
Requesting for lattitude = 8.0 and longitude = 72.0
Requesting for lattitude = 8.0 and longitude = 73.0
Requesting for lattitude = 8.0 and longitude = 74.0
Requesting for lattitude = 8.0 and longitude = 75.0
Requesting for lattitude = 8.0 and longitude = 76.0
Requesting for lattitude = 8.0 and longitude = 77.0
Requesting for lattitude = 8.0 and longitude = 78.0
Requesting for lattitude = 8.0 and longitude = 79.0
Requesting for lattitude = 8.0 and longitude = 80.0
Requesting for lattitude = 8.0 and longitude = 81.0
Requesting for lattitude = 8.0 and longitude = 82.0
Requesting for lattitude = 8.0 and longitude = 83.0
Requesting for lattitude = 8.0 and longitude = 84.0
Requesting for lattitude = 8.0 and long

,PVOUT_csi,DNI,GHI,DIF,GTI_opta,OPTA,TEMP,ELE


In [35]:

solar = pd.read_csv('solar_test.csv')

In [36]:
solar

,lat,long,PVOUT_csi (kWh/kWp),DNI (kWh/m2),GHI (kWh/m2),DIF (kWh/m2),GTI_opta (kWh/m2),OPTA (°),TEMP (°C),ELE (m)
0,16,72,1652.988159,1496.078125,1991.875000,916.343750,2087.539062,19,27.5000,-1483
1,18,73,1586.837036,1440.335938,1918.757812,901.554688,2027.250000,21,24.4375,560
2,17,74,1594.165039,1417.109375,1919.765625,911.484375,2029.765625,21,27.0000,-7
3,16,73,1625.696411,1429.109375,1955.132812,924.218750,2050.531250,19,27.3750,-73
4,17,73,1617.866577,1420.664062,1938.507812,919.867188,2041.429688,20,27.1250,-49
5,18,72,1655.563843,1512.914062,1978.929688,905.656250,2090.632812,21,27.2500,-39
6,17,72,1652.425781,1501.640625,1984.023438,912.062500,2088.210938,20,27.4375,-167
7,18,74,1596.979248,1464.375000,1923.695312,897.421875,2038.437500,22,23.5625,832
8,16,74,1513.380249,1172.828125,1712.750000,832.664062,1777.085938,22,23.3750,463


In [102]:
solar.describe()

,lat,long,PVOUT_csi (kWh/kWp),DNI (kWh/m2),GHI (kWh/m2),DIF (kWh/m2),GTI_opta (kWh/m2),OPTA (°),TEMP (°C),ELE (m)
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,20.500000,77.500000,1598.020365,1459.697917,1918.679253,904.672743,2056.091363,23.722222,26.614583,393.083333
std,1.732051,1.732051,16.699905,55.374802,17.759104,31.757937,19.884796,1.666667,0.937143,144.341732
min,18.000000,75.000000,1568.619141,1344.031250,1886.500000,848.367188,2018.093750,21.000000,24.687500,125.000000
25%,19.000000,76.000000,1587.621490,1424.193359,1905.027344,880.289062,2047.175781,22.750000,25.812500,276.500000
50%,20.500000,77.500000,1594.717896,1452.351562,1914.382812,910.003906,2054.285156,24.000000,26.750000,365.000000
75%,22.000000,79.000000,1609.255920,1495.955078,1930.785156,925.847656,2067.316406,25.000000,27.375000,510.500000
max,23.000000,80.000000,1641.278320,1597.343750,1948.062500,963.343750,2113.476562,26.000000,28.250000,712.000000


In [103]:
solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   lat                   36 non-null     int64  
 1   long                  36 non-null     int64  
 2    PVOUT_csi (kWh/kWp)  36 non-null     float64
 3    DNI (kWh/m2)         36 non-null     float64
 4    GHI (kWh/m2)         36 non-null     float64
 5    DIF (kWh/m2)         36 non-null     float64
 6    GTI_opta (kWh/m2)    36 non-null     float64
 7    OPTA (°)             36 non-null     int64  
 8    TEMP (°C)            36 non-null     float64
 9    ELE (m)              36 non-null     int64  
dtypes: float64(6), int64(4)
memory usage: 2.9 KB
